In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
from functools import reduce

# Helper Functions

In [ ]:
def clean_char(df):
    
    special_char = ['\xad','À','Á','Ç','É','Ñ','Ó','Ö','Ø','Ü','ß',
                     'à','á','â','ã','ä','å','æ','ç','è','é','ê','ë',
                     'í','î','ï','ð','ñ','ò','ó','ô','ö','ø','ú','ü',
                     'ý','ă','ą','Ć','ć','Č','č','Đ','đ','ę','ě','ğ',
                     'İ','ı','ľ','Ł','ł','ń','ň','ō','ř','Ş','ş','Š',
                     'š','ů','Ž','ž','Ș','ș','ț']

    replacement = ['','A','A','C','E','N','O','O','O','U','ss',
                    'a','a','a','a','a','a','ae','c','e','e','e',
                    'e','i','i','i','d','n','o','o','o','o','o',
                    'u','u','y','a','a','C','c','C','c','D','d',
                    'e','e','g','I','i','l','L','l','n','n','o',
                    'r','S','s','S','s','u','Z','z','S','s','t']



    #Replacing Special characters in the 'Player' Column of the fbref dataframe.

    clean_players = []
    for a in df['Player']:

        string = a

        player_string = []
        for z in string:
            if z in special_char:
                for e in list(range(len(special_char))):
                    if z == special_char[e]:
                        player_string.append(replacement[e])
                    else:
                        pass
            else:
                player_string.append(z)
        player_string = ''.join(player_string)
        clean_players.append(player_string)



    df['Player'] = clean_players 
    return df


#---------------------------------------------------------------------------------------------------


def clean_std(dataframe):

    original = []
    duplicate = []

    for a in dataframe['Player']:
        if a not in original:
            original.append(a)
        else:
            duplicate.append(a)
    
    mid_season_transfers = pd.DataFrame(columns = dataframe.columns)

    for a in duplicate:
        player = dataframe[dataframe['Player']==a]
        mid_season_transfers = mid_season_transfers.append(dataframe[dataframe['Player']==a])
    
    
    first = mid_season_transfers.iloc[::2]
    second = mid_season_transfers.iloc[1::2]
    
    midseason = []

    for a in list(range(len(first))):

        double_club_combined = pd.Series([first.iloc[a]['Player'],
                           [first.iloc[a]['Squad'],second.iloc[a]['Squad']],
                           first.iloc[a]['MP']+second.iloc[a]['MP'],
                            first.iloc[a]['Starts']+second.iloc[a]['Starts'],
                            first.iloc[a]['Min']+second.iloc[a]['Min'],
                           (first.iloc[a]['Min']+second.iloc[a]['Min'])/90,
                           first.iloc[a]['Gls']+second.iloc[a]['Gls'],
                           first.iloc[a]['Ast']+second.iloc[a]['Ast'],
                           first.iloc[a]['Non-Penalty Goals']+second.iloc[a]['Non-Penalty Goals'],
                           first.iloc[a]['Penalties Scored']+second.iloc[a]['Penalties Scored'],
                           first.iloc[a]['Penalties Attempted']+second.iloc[a]['Penalties Attempted'],
                           first.iloc[a]['Yellow Cards']+second.iloc[a]['Yellow Cards'],
                           first.iloc[a]['Red Cards']+second.iloc[a]['Red Cards'],
                           (first.iloc[a]['Gls']+second.iloc[a]['Gls'])/90,
                           (first.iloc[a]['Ast']+second.iloc[a]['Ast'])/90,
                           ((first.iloc[a]['Gls']+second.iloc[a]['Gls'])+(first.iloc[a]['Ast']+second.iloc[a]['Ast']))/90,
                           ((first.iloc[a]['Gls']+second.iloc[a]['Gls'])-(first.iloc[a]['Penalties Scored']+second.iloc[a]['Penalties Scored']))/90,
                           ((first.iloc[a]['Gls']+second.iloc[a]['Gls'])+(first.iloc[a]['Ast']+second.iloc[a]['Ast'])-(first.iloc[a]['Penalties Scored']+second.iloc[a]['Penalties Scored']))/90,
                           first.iloc[a]['xG']+second.iloc[a]['xG'],
                           first.iloc[a]['Non-Penalty xG']+second.iloc[a]['Non-Penalty xG'],
                           first.iloc[a]['xA']+second.iloc[a]['xA'],
                           first.iloc[a]['Non-penalty xG+ xA']+second.iloc[a]['Non-penalty xG+ xA'],
                           (first.iloc[a]['xG']+second.iloc[a]['xG'])/90,
                           (first.iloc[a]['xA']+second.iloc[a]['xA'])/90,
                           ((first.iloc[a]['xG']+second.iloc[a]['xG'])+(first.iloc[a]['xA']+second.iloc[a]['xA']))/90,
                           (first.iloc[a]['Non-Penalty xG']+second.iloc[a]['Non-Penalty xG'])/90,
                           (first.iloc[a]['Non-penalty xG+ xA']+second.iloc[a]['Non-penalty xG+ xA'])/90])

        midseason.append(double_club_combined)

    transferred_players = pd.DataFrame(midseason)
    transferred_players.columns = ['Player','Squad','MP','Starts','Min',
                                   'Total Mins/90','Gls','Ast','Non-Penalty Goals',
                                    'Penalties Scored','Penalties Attempted','Yellow Cards',
                                   'Red Cards','Gls/90','Ast/90','(G+A)/90','Non-Penalty Goals/90',
                                   '(Gls+Ast-Scored Penalties)/90','xG','Non-Penalty xG','xA',
                                    'Non-penalty xG+ xA','xG/90','xA/90','(xG+xA)/90',
                                     'Non-Penalty xG/90','(Non-Penalty xG+xA)/90']
    
    final_df = pd.concat([dataframe, mid_season_transfers, mid_season_transfers]).drop_duplicates(keep=False)
    final_df = pd.concat([final_df,transferred_players])
    return final_df
    

#---------------------------------------------------------------------------------------------------
    
def check_dupes(dataframe):

    original = []
    duplicate = []

    for a in dataframe['Player']:
        if a not in original:
            original.append(a)
        else:
            duplicate.append(a)

    mid_season_transfers = pd.DataFrame(columns = dataframe.columns)

    for a in duplicate:
        player = dataframe[dataframe['Player']==a]
        mid_season_transfers = mid_season_transfers.append(dataframe[dataframe['Player']==a])
    
    return mid_season_transfers

# Importing Data

In [3]:
#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
england_std21 = pd.read_excel('data/league_data/england/21/england_std.xlsx')
england_shooting21 = pd.read_excel('data/league_data/england/21/england_shooting.xlsx')
england_shot_creation21 = pd.read_excel('data/league_data/england/21/england_shot_creation.xlsx')
england_possession21 = pd.read_excel('data/league_data/england/21/england_possession.xlsx')
england_passing21 = pd.read_excel('data/league_data/england/21/england_passing.xlsx')
england_defend21 = pd.read_excel('data/league_data/england/21/england_defending.xlsx')
england_misc21 = pd.read_excel('data/league_data/england/21/england_misc.xlsx')



std_cols = england_std21.columns
shooting_cols = england_shooting21.columns
shot_creation_cols = england_shot_creation21.columns
possession_cols = england_possession21.columns
passing_cols = england_passing21.columns
defending_cols = england_defend21.columns
misc_cols = england_misc21.columns



#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
england_std20 = pd.read_excel('data/league_data/england/20/england_std.xlsx')
england_std20.columns = std_cols

england_shooting20 = pd.read_excel('data/league_data/england/20/england_shooting.xlsx')
england_shooting20.columns = shooting_cols


england_shot_creation20 = pd.read_excel('data/league_data/england/20/england_shot_creation.xlsx')
england_shot_creation20.columns = shot_creation_cols


england_possession20 = pd.read_excel('data/league_data/england/20/england_possession.xlsx')
england_possession20.columns = possession_cols


england_passing20 = pd.read_excel('data/league_data/england/20/england_passing.xlsx')
england_passing20.columns = passing_cols


england_defend20 = pd.read_excel('data/league_data/england/20/england_defending.xlsx')
england_defend20.columns = defending_cols


england_misc20 = pd.read_excel('data/league_data/england/20/england_misc.xlsx')
england_misc20.columns = misc_cols





#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
england_std19 = pd.read_excel('data/league_data/england/19/england_std.xlsx')
england_std19.columns = std_cols

england_shooting19 = pd.read_excel('data/league_data/england/19/england_shooting.xlsx')
england_shooting19.columns = shooting_cols


england_shot_creation19 = pd.read_excel('data/league_data/england/19/england_shot_creation.xlsx')
england_shot_creation19.columns = shot_creation_cols


england_possession19 = pd.read_excel('data/league_data/england/19/england_possession.xlsx')
england_possession19.columns = possession_cols


england_passing19 = pd.read_excel('data/league_data/england/19/england_passing.xlsx')
england_passing19.columns = passing_cols


england_defend19 = pd.read_excel('data/league_data/england/19/england_defending.xlsx')
england_defend19.columns = defending_cols


england_misc19 = pd.read_excel('data/league_data/england/19/england_misc.xlsx')
england_misc19.columns = misc_cols




#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
england_std18 = pd.read_excel('data/league_data/england/18/england_std.xlsx')
england_std18.columns = std_cols

england_shooting18 = pd.read_excel('data/league_data/england/18/england_shooting.xlsx')
england_shooting18.columns = shooting_cols


england_shot_creation18 = pd.read_excel('data/league_data/england/18/england_shot_creation.xlsx')
england_shot_creation18.columns = shot_creation_cols


england_possession18 = pd.read_excel('data/league_data/england/18/england_possession.xlsx')
england_possession18.columns = possession_cols


england_passing18 = pd.read_excel('data/league_data/england/18/england_passing.xlsx')
england_passing18.columns = passing_cols


england_defend18 = pd.read_excel('data/league_data/england/18/england_defending.xlsx')
england_defend18.columns = defending_cols


england_misc18 = pd.read_excel('data/league_data/england/18/england_misc.xlsx')
england_misc18.columns = misc_cols




#---------------------------------------------------------------------------------------------------


#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
france_std21 = pd.read_excel('data/league_data/france/21/france_std.xlsx')
france_std21.columns = std_cols

france_shooting21 = pd.read_excel('data/league_data/france/21/france_shooting.xlsx')
france_shooting21.columns = shooting_cols


france_shot_creation21 = pd.read_excel('data/league_data/france/21/france_shot_creation.xlsx')
france_shot_creation21.columns = shot_creation_cols

france_possession21 = pd.read_excel('data/league_data/france/21/france_possession.xlsx')
france_possession21.columns = possession_cols


france_passing21 = pd.read_excel('data/league_data/france/21/france_passing.xlsx')
france_passing21.columns = passing_cols


france_defend21 = pd.read_excel('data/league_data/france/21/france_defending.xlsx')
france_defend21.columns = defending_cols


france_misc21 = pd.read_excel('data/league_data/france/21/france_misc.xlsx')
france_misc21.columns = misc_cols





#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
france_std20 = pd.read_excel('data/league_data/france/20/france_std.xlsx')
france_std20.columns = std_cols

france_shooting20 = pd.read_excel('data/league_data/france/20/france_shooting.xlsx')
france_shooting20.columns = shooting_cols


france_shot_creation20 = pd.read_excel('data/league_data/france/20/france_shot_creation.xlsx')
france_shot_creation20.columns = shot_creation_cols


france_possession20 = pd.read_excel('data/league_data/france/20/france_possession.xlsx')
france_possession20.columns = possession_cols


france_passing20 = pd.read_excel('data/league_data/france/20/france_passing.xlsx')
france_passing20.columns = passing_cols


france_defend20 = pd.read_excel('data/league_data/france/20/france_defending.xlsx')
france_defend20.columns = defending_cols


france_misc20 = pd.read_excel('data/league_data/france/20/france_misc.xlsx')
france_misc20.columns = misc_cols



#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
france_std19 = pd.read_excel('data/league_data/france/19/france_std.xlsx')
france_std19.columns = std_cols

france_shooting19 = pd.read_excel('data/league_data/france/19/france_shooting.xlsx')
france_shooting19.columns = shooting_cols


france_shot_creation19 = pd.read_excel('data/league_data/france/19/france_shot_creation.xlsx')
france_shot_creation19.columns = shot_creation_cols


france_possession19 = pd.read_excel('data/league_data/france/19/france_possession.xlsx')
france_possession19.columns = possession_cols


france_passing19 = pd.read_excel('data/league_data/france/19/france_passing.xlsx')
france_passing19.columns = passing_cols


france_defend19 = pd.read_excel('data/league_data/france/19/france_defending.xlsx')
france_defend19.columns = defending_cols


france_misc19 = pd.read_excel('data/league_data/france/19/france_misc.xlsx')
france_misc19.columns = misc_cols




#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
france_std18 = pd.read_excel('data/league_data/france/18/france_std.xlsx')
france_std18.columns = std_cols

france_shooting18 = pd.read_excel('data/league_data/france/18/france_shooting.xlsx')
france_shooting18.columns = shooting_cols


france_shot_creation18 = pd.read_excel('data/league_data/france/18/france_shot_creation.xlsx')
france_shot_creation18.columns = shot_creation_cols


france_possession18 = pd.read_excel('data/league_data/france/18/france_possession.xlsx')
france_possession18.columns = possession_cols


france_passing18 = pd.read_excel('data/league_data/france/18/france_passing.xlsx')
france_passing18.columns = passing_cols


france_defend18 = pd.read_excel('data/league_data/france/18/france_defending.xlsx')
france_defend18.columns = defending_cols


france_misc18 = pd.read_excel('data/league_data/france/18/france_misc.xlsx')
france_misc18.columns = misc_cols


#---------------------------------------------------------------------------------------------------

#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
germany_std21 = pd.read_excel('data/league_data/germany/21/germany_std.xlsx')
germany_std21.columns = std_cols

germany_shooting21 = pd.read_excel('data/league_data/germany/21/germany_shooting.xlsx')
germany_shooting21.columns = shooting_cols


germany_shot_creation21 = pd.read_excel('data/league_data/germany/21/germany_shot_creation.xlsx')
germany_shot_creation21.columns = shot_creation_cols

germany_possession21 = pd.read_excel('data/league_data/germany/21/germany_possession.xlsx')
germany_possession21.columns = possession_cols


germany_passing21 = pd.read_excel('data/league_data/germany/21/germany_passing.xlsx')
germany_passing21.columns = passing_cols


germany_defend21 = pd.read_excel('data/league_data/germany/21/germany_defending.xlsx')
germany_defend21.columns = defending_cols


germany_misc21 = pd.read_excel('data/league_data/germany/21/germany_misc.xlsx')
germany_misc21.columns = misc_cols







#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
germany_std20 = pd.read_excel('data/league_data/germany/20/germany_std.xlsx')
germany_std20.columns = std_cols

germany_shooting20 = pd.read_excel('data/league_data/germany/20/germany_shooting.xlsx')
germany_shooting20.columns = shooting_cols


germany_shot_creation20 = pd.read_excel('data/league_data/germany/20/germany_shot_creation.xlsx')
germany_shot_creation20.columns = shot_creation_cols


germany_possession20 = pd.read_excel('data/league_data/germany/20/germany_possession.xlsx')
germany_possession20.columns = possession_cols


germany_passing20 = pd.read_excel('data/league_data/germany/20/germany_passing.xlsx')
germany_passing20.columns = passing_cols


germany_defend20 = pd.read_excel('data/league_data/germany/20/germany_defending.xlsx')
germany_defend20.columns = defending_cols


germany_misc20 = pd.read_excel('data/league_data/germany/20/germany_misc.xlsx')
germany_misc20.columns = misc_cols





#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
germany_std19 = pd.read_excel('data/league_data/germany/19/germany_std.xlsx')
germany_std19.columns = std_cols

germany_shooting19 = pd.read_excel('data/league_data/germany/19/germany_shooting.xlsx')
germany_shooting19.columns = shooting_cols


germany_shot_creation19 = pd.read_excel('data/league_data/germany/19/germany_shot_creation.xlsx')
germany_shot_creation19.columns = shot_creation_cols


germany_possession19 = pd.read_excel('data/league_data/germany/19/germany_possession.xlsx')
germany_possession19.columns = possession_cols


germany_passing19 = pd.read_excel('data/league_data/germany/19/germany_passing.xlsx')
germany_passing19.columns = passing_cols


germany_defend19 = pd.read_excel('data/league_data/germany/19/germany_defending.xlsx')
germany_defend19.columns = defending_cols


germany_misc19 = pd.read_excel('data/league_data/germany/19/germany_misc.xlsx')
germany_misc19.columns = misc_cols



#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
germany_std18 = pd.read_excel('data/league_data/germany/18/germany_std.xlsx')
germany_std18.columns = std_cols

germany_shooting18 = pd.read_excel('data/league_data/germany/18/germany_shooting.xlsx')
germany_shooting18.columns = shooting_cols


germany_shot_creation18 = pd.read_excel('data/league_data/germany/18/germany_shot_creation.xlsx')
germany_shot_creation18.columns = shot_creation_cols


germany_possession18 = pd.read_excel('data/league_data/germany/18/germany_possession.xlsx')
germany_possession18.columns = possession_cols


germany_passing18 = pd.read_excel('data/league_data/germany/18/germany_passing.xlsx')
germany_passing18.columns = passing_cols


germany_defend18 = pd.read_excel('data/league_data/germany/18/germany_defending.xlsx')
germany_defend18.columns = defending_cols


germany_misc18 = pd.read_excel('data/league_data/germany/18/germany_misc.xlsx')
germany_misc18.columns = misc_cols



#---------------------------------------------------------------------------------------------------


#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
italy_std21 = pd.read_excel('data/league_data/italy/21/italy_std.xlsx')
italy_std21.columns = std_cols

italy_shooting21 = pd.read_excel('data/league_data/italy/21/italy_shooting.xlsx')
italy_shooting21.columns = shooting_cols


italy_shot_creation21 = pd.read_excel('data/league_data/italy/21/italy_shot_creation.xlsx')
italy_shot_creation21.columns = shot_creation_cols

italy_possession21 = pd.read_excel('data/league_data/italy/21/italy_possession.xlsx')
italy_possession21.columns = possession_cols


italy_passing21 = pd.read_excel('data/league_data/italy/21/italy_passing.xlsx')
italy_passing21.columns = passing_cols


italy_defend21 = pd.read_excel('data/league_data/italy/21/italy_defending.xlsx')
italy_defend21.columns = defending_cols


italy_misc21 = pd.read_excel('data/league_data/italy/21/italy_misc.xlsx')
italy_misc21.columns = misc_cols


#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
italy_std20 = pd.read_excel('data/league_data/italy/20/italy_std.xlsx')
italy_std20.columns = std_cols

italy_shooting20 = pd.read_excel('data/league_data/italy/20/italy_shooting.xlsx')
italy_shooting20.columns = shooting_cols


italy_shot_creation20 = pd.read_excel('data/league_data/italy/20/italy_shot_creation.xlsx')
italy_shot_creation20.columns = shot_creation_cols


italy_possession20 = pd.read_excel('data/league_data/italy/20/italy_possession.xlsx')
italy_possession20.columns = possession_cols


italy_passing20 = pd.read_excel('data/league_data/italy/20/italy_passing.xlsx')
italy_passing20.columns = passing_cols


italy_defend20 = pd.read_excel('data/league_data/italy/20/italy_defending.xlsx')
italy_defend20.columns = defending_cols


italy_misc20 = pd.read_excel('data/league_data/italy/20/italy_misc.xlsx')
italy_misc20.columns = misc_cols


#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
italy_std19 = pd.read_excel('data/league_data/italy/19/italy_std.xlsx')
italy_std19.columns = std_cols

italy_shooting19 = pd.read_excel('data/league_data/italy/19/italy_shooting.xlsx')
italy_shooting19.columns = shooting_cols


italy_shot_creation19 = pd.read_excel('data/league_data/italy/19/italy_shot_creation.xlsx')
italy_shot_creation19.columns = shot_creation_cols


italy_possession19 = pd.read_excel('data/league_data/italy/19/italy_possession.xlsx')
italy_possession19.columns = possession_cols


italy_passing19 = pd.read_excel('data/league_data/italy/19/italy_passing.xlsx')
italy_passing19.columns = passing_cols


italy_defend19 = pd.read_excel('data/league_data/italy/19/italy_defending.xlsx')
italy_defend19.columns = defending_cols


italy_misc19 = pd.read_excel('data/league_data/italy/19/italy_misc.xlsx')
italy_misc19.columns = misc_cols


#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
italy_std18 = pd.read_excel('data/league_data/italy/18/italy_std.xlsx')
italy_std18.columns = std_cols

italy_shooting18 = pd.read_excel('data/league_data/italy/18/italy_shooting.xlsx')
italy_shooting18.columns = shooting_cols


italy_shot_creation18 = pd.read_excel('data/league_data/italy/18/italy_shot_creation.xlsx')
italy_shot_creation18.columns = shot_creation_cols


italy_possession18 = pd.read_excel('data/league_data/italy/18/italy_possession.xlsx')
italy_possession18.columns = possession_cols


italy_passing18 = pd.read_excel('data/league_data/italy/18/italy_passing.xlsx')
italy_passing18.columns = passing_cols


italy_defend18 = pd.read_excel('data/league_data/italy/18/italy_defending.xlsx')
italy_defend18.columns = defending_cols


italy_misc18 = pd.read_excel('data/league_data/italy/18/italy_misc.xlsx')
italy_misc18.columns = misc_cols



#---------------------------------------------------------------------------------------------------


#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
spain_std21 = pd.read_excel('data/league_data/spain/21/spain_std.xlsx')
spain_std21.columns = std_cols

spain_shooting21 = pd.read_excel('data/league_data/spain/21/spain_shooting.xlsx')
spain_shooting21.columns = shooting_cols


spain_shot_creation21 = pd.read_excel('data/league_data/spain/21/spain_shot_creation.xlsx')
spain_shot_creation21.columns = shot_creation_cols


spain_possession21 = pd.read_excel('data/league_data/spain/21/spain_possession.xlsx')
spain_possession21.columns = possession_cols


spain_passing21 = pd.read_excel('data/league_data/spain/21/spain_passing.xlsx')
spain_passing21.columns = passing_cols


spain_defend21 = pd.read_excel('data/league_data/spain/21/spain_defending.xlsx')
spain_defend21.columns = defending_cols


spain_misc21 = pd.read_excel('data/league_data/spain/21/spain_misc.xlsx')
spain_misc21.columns = misc_cols




#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
spain_std20 = pd.read_excel('data/league_data/spain/20/spain_std.xlsx')
spain_std20.columns = std_cols

spain_shooting20 = pd.read_excel('data/league_data/spain/20/spain_shooting.xlsx')
spain_shooting20.columns = shooting_cols


spain_shot_creation20 = pd.read_excel('data/league_data/spain/20/spain_shot_creation.xlsx')
spain_shot_creation20.columns = shot_creation_cols


spain_possession20 = pd.read_excel('data/league_data/spain/20/spain_possession.xlsx')
spain_possession20.columns = possession_cols


spain_passing20 = pd.read_excel('data/league_data/spain/20/spain_passing.xlsx')
spain_passing20.columns = passing_cols


spain_defend20 = pd.read_excel('data/league_data/spain/20/spain_defending.xlsx')
spain_defend20.columns = defending_cols


spain_misc20 = pd.read_excel('data/league_data/spain/20/spain_misc.xlsx')
spain_misc20.columns = misc_cols



#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
spain_std19 = pd.read_excel('data/league_data/spain/19/spain_std.xlsx')
spain_std19.columns = std_cols

spain_shooting19 = pd.read_excel('data/league_data/spain/19/spain_shooting.xlsx')
spain_shooting19.columns = shooting_cols


spain_shot_creation19 = pd.read_excel('data/league_data/spain/19/spain_shot_creation.xlsx')
spain_shot_creation19.columns = shot_creation_cols


spain_possession19 = pd.read_excel('data/league_data/spain/19/spain_possession.xlsx')
spain_possession19.columns = possession_cols

spain_passing19 = pd.read_excel('data/league_data/spain/19/spain_passing.xlsx')
spain_passing19.columns = passing_cols


spain_defend19 = pd.read_excel('data/league_data/spain/19/spain_defending.xlsx')
spain_defend19.columns = defending_cols


spain_misc19 = pd.read_excel('data/league_data/spain/19/spain_misc.xlsx')
spain_misc19.columns = misc_cols



#Importing English Premier League Data data. The column names were modified by me in MS Excel before importing them here.
spain_std18 = pd.read_excel('data/league_data/spain/18/spain_std.xlsx')
spain_std18.columns = std_cols

spain_shooting18 = pd.read_excel('data/league_data/spain/18/spain_shooting.xlsx')
spain_shooting18.columns = shooting_cols


spain_shot_creation18 = pd.read_excel('data/league_data/spain/18/spain_shot_creation.xlsx')
spain_shot_creation18.columns = shot_creation_cols


spain_possession18 = pd.read_excel('data/league_data/spain/18/spain_possession.xlsx')
spain_possession18.columns = possession_cols


spain_passing18 = pd.read_excel('data/league_data/spain/18/spain_passing.xlsx')
spain_passing18.columns = passing_cols


spain_defend18 = pd.read_excel('data/league_data/spain/18/spain_defending.xlsx')
spain_defend18.columns = defending_cols


spain_misc18 = pd.read_excel('data/league_data/spain/18/spain_misc.xlsx')
spain_misc18.columns = misc_cols

In [13]:
clean_char(spain_std21)[clean_char(spain_std21)['Squad']=='Osasuna']

,Player,Squad,MP,Starts,Min,Total Mins/90,Gls,Ast,Non-Penalty Goals,Penalties Scored,...,(Gls+Ast-Scored Penalties)/90,xG,Non-Penalty xG,xA,Non-penalty xG+ xA,xG/90,xA/90,(xG+xA)/90,Non-Penalty xG/90,(Non-Penalty xG+xA)/90
8,David Garcia,Osasuna,35,35,3150,35.0,1,0,1,0,...,0.03,1.5,1.5,0.3,1.8,0.04,0.01,0.05,0.04,0.05
26,Sergio Herrera,Osasuna,33,32,2958,32.9,0,0,0,0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
51,Ruben Garcia,Osasuna,37,33,2718,30.2,3,5,2,1,...,0.23,3.4,2.7,5.1,7.8,0.11,0.17,0.28,0.09,0.26
62,Jon Moncayola,Osasuna,36,28,2660,29.6,2,0,2,0,...,0.07,1.5,1.5,1.3,2.7,0.05,0.04,0.09,0.05,0.09
110,Nacho Vidal,Osasuna,34,25,2348,26.1,0,0,0,0,...,0.00,0.5,0.5,1.7,2.2,0.02,0.07,0.08,0.02,0.08
150,Oier Sanjurjo,Osasuna,32,24,2084,23.2,0,2,0,0,...,0.09,0.9,0.9,1.8,2.6,0.04,0.08,0.11,0.04,0.11
161,Aridane Hernandez,Osasuna,24,24,2026,22.5,0,0,0,0,...,0.00,0.1,0.1,0.5,0.6,0.00,0.02,0.03,0.00,0.03
171,Juan Cruz Armada,Osasuna,26,23,1962,21.8,0,0,0,0,...,0.00,0.0,0.0,0.5,0.5,0.00,0.02,0.02,0.00,0.02
181,Roberto Torres,Osasuna,35,19,1894,21.0,7,1,3,4,...,0.19,6.2,2.4,1.5,3.9,0.29,0.07,0.37,0.11,0.18
213,Ante Budimir,Osasuna,30,18,1625,18.1,11,0,11,0,...,0.61,7.1,7.1,0.6,7.7,0.39,0.03,0.43,0.39,0.43


In [4]:
s21_std = pd.concat([england_std21,france_std21,germany_std21,italy_std21,spain_std21])

In [8]:
def check_dupes(dataframe):

    original = []
    duplicate = []

    for a in dataframe['Player']:
        if a not in original:
            original.append(a)
        else:
            duplicate.append(a)

    mid_season_transfers = pd.DataFrame(columns = dataframe.columns)

    for a in duplicate:
        player = dataframe[dataframe['Player']==a]
        mid_season_transfers = mid_season_transfers.append(dataframe[dataframe['Player']==a])
    
    return mid_season_transfers